In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/diegogomez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/diegogomez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Import the Dataset

En este paso se importan los dos archivos de datos desde la carpeta `./docs`.
Luego se crea una nueva columna en cada Dataframe que representa el documento completo
(Titulo + contenido). Se le hace `drop` al resto de columnas.
El arreglo `targert` se construye a partir de los archivos anteriores, la idea es que sea un arreglo
de dimensión (n,1) donde n es la suma de el tamaño de los documentos anteriores. Además, los primeros k valores
se marcarán con 1, donde k es el tamaño de los datos de noticias reales, el resto son ceros representando las falsas.


In [2]:
datos_true = pd.read_csv('data/pos/True.csv')
datos_false = pd.read_csv('data/neg/Fake.csv')

datos_true['documento'] = datos_true['title']+ datos_true['text']
datos_false['documento'] = datos_false['title'] + datos_false['text']

target_true = np.ones(len(datos_true['documento']))
target_false = np.zeros(len(datos_false['documento']))
target = np.concatenate((target_true, target_false))
# target = target.reshape((len(target), 1))
datos_true.drop(columns=['title', 'text', 'subject', 'date'], axis=1, inplace=True)
datos_false.drop(columns=['title', 'text', 'subject', 'date'], axis=1, inplace=True)

#datos = load_files(container_path='./data')
#X = []
print(target)
print(len(target))

[1. 1. 1. ... 0. 0. 0.]
44898


## Preprocesamiento

Se le aplica a cada Dataframe un filtrado de caracteres con poco valor para el modelo.

In [3]:
stemmer = WordNetLemmatizer()
def regex_filter(text):
        # Remove all the special characters
    document = re.sub(r'\W', ' ', text)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document

datos_true['documento'].apply(regex_filter)
datos_false['documento'].apply(regex_filter)

print(datos_true['documento'])

0        As U.S. budget fight looms, Republicans flip t...
1        U.S. military to accept transgender recruits o...
2        Senior U.S. Republican senator: 'Let Mr. Muell...
3        FBI Russia probe helped by Australian diplomat...
4        Trump wants Postal Service to charge 'much mor...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorit...
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: documento, Length: 21417, dtype: object


## Text to number
Se pasan las columnas con los datos relevantes a una lista y se concatenan
 con la de noticias ciertas primero (súper importante por cómo se contruyó el arreglo `Target`).
Luego se vectoriza la información de texto que pasamos. Los parámetros representan lo siguiente:
`max_feature` es el límite de palabras a vectorizas, es decir, solo se tendrán en cuenta las 2000 palabras
más frecuentes en este caso. `min_df` es la cantidad mínima de datos que debe contener una palabra para que
se tenga en cuenta. `max_df` es el porcentaje másximo de datos que pueden contener una palabra para que se tenga en cuenta.
En este caso descartamos las palabras que aparezcan en más del 70% de datos. `Stop word` es el diccionario
para filtrar palabras vacías.

In [4]:
documents = datos_true['documento'].to_list() + datos_false['documento'].to_list()
vectorizer = CountVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

El resultado es una matríz donde cada fila tiene 2000 entradas (1 por palabra encontrada) y el número de filas es
igual al número de datos (Noticias verdaderas + noticias falsas)

In [5]:
print(vectorizer.vocabulary_)

{'budget': 283, 'fight': 687, 'republicans': 1519, 'fiscal': 706, 'reuters': 1543, 'head': 813, 'conservative': 431, 'republican': 1518, 'congress': 427, 'voted': 1916, 'month': 1154, 'huge': 853, 'national': 1181, 'debt': 510, 'pay': 1294, 'tax': 1786, 'cuts': 493, 'called': 296, 'sunday': 1757, 'urged': 1886, '2018': 24, 'keeping': 971, 'way': 1938, 'among': 131, 'representative': 1515, 'mark': 1091, 'speaking': 1699, 'cbs': 328, 'face': 653, 'nation': 1180, 'drew': 574, 'hard': 810, 'line': 1045, 'federal': 681, 'spending': 1705, 'lawmakers': 1007, 'battle': 221, 'january': 943, 'return': 1540, 'wednesday': 1944, 'begin': 226, 'trying': 1862, 'pass': 1289, 'likely': 1042, 'linked': 1047, 'issues': 938, 'immigration': 872, 'policy': 1333, 'even': 629, 'november': 1216, 'congressional': 428, 'election': 598, 'campaigns': 302, 'approach': 154, 'seek': 1611, 'keep': 970, 'control': 443, 'president': 1366, 'donald': 566, 'trump': 1858, 'want': 1926, 'big': 245, 'increase': 884, 'military

## Training and Testing Sets
Divide el conjunto con los datos `X`y la variable `target`. `test_size` representa el tamaño del set de prueba

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.4, random_state=10)

In [7]:
print(X_train.shape)
print(X_test.shape)
print(26938+17960)

(26938, 2000)
(17960, 2000)
44898


## Training Text Classification Model
Aplicación del algoritmo deseado, en este caso un RandomForest

In [15]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

## Evaluating

In [18]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))




[[9388   17]
 [   9 8546]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      9405
         1.0       1.00      1.00      1.00      8555

    accuracy                           1.00     17960
   macro avg       1.00      1.00      1.00     17960
weighted avg       1.00      1.00      1.00     17960

0.9985523385300669


## KNN a ver qué pasa

In [12]:
knn_model = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)

## Evaluating KNN


In [13]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[8443  962]
 [2621 5934]]
              precision    recall  f1-score   support

         0.0       0.76      0.90      0.82      9405
         1.0       0.86      0.69      0.77      8555

    accuracy                           0.80     17960
   macro avg       0.81      0.80      0.80     17960
weighted avg       0.81      0.80      0.80     17960

0.800501113585746


## Saving Model

In [14]:
with open('text_classifier_k_4', 'wb') as picklefile:
    pickle.dump(knn_model,picklefile)